In [1]:
import argparse
from equilibrator_api import ComponentContribution, ReactionMatcher, CompoundMatcher, ParseError, Pathway
from equilibrator_api.bounds import Bounds
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
reaction_matcher = ReactionMatcher()

def estimate_dG0(plaintext, pH=7.0, I=0.1):
    # parse the reaction
    try:
        reaction = reaction_matcher.match(plaintext)
    except ParseError:
        print('incomplete formula')
        return
    except ValueError as e:
        print(e)
        return

    html = '<p>pH = %.2f, I = %.2f M</br>KEGG reaction: %s</br></p>' % (pH, I, reaction.write_formula())
    
    equilibrator = ComponentContribution(pH=pH, ionic_strength=I)
    
    n_e = reaction.check_half_reaction_balancing()
    if n_e is None:
        html += '<p>ERROR: reaction is not chemically balanced</p>'
    elif n_e == 0:
        dG0_prime, dG0_uncertainty = equilibrator.dG0_prime(reaction)
        html += "<h2>\u0394G\'\u00B0 = %.2f \u00B1 %.2f kJ/mol</h2>" % (dG0_prime, 1.96 * dG0_uncertainty)
        ln_RI = equilibrator.reversibility_index(reaction)
        html += '<p>ln(Reversibility Index) = %.1f</p>' % ln_RI

    else:  # treat as a half-reaction
        E0_prime_mV, E0_uncertainty = equilibrator.E0_prime(reaction)
        html += '<h2>E\'\u00B0 = %.1f \u00B1 %.1f mV</h2>' % (E0_prime_mV, E0_uncertainty)

    w = widgets.HTML(html)
    display(w)
        

In [5]:
interact(estimate_dG0,
         plaintext=widgets.Text(value='C00022 = C00084 + C00011', font_size=20),
         pH=widgets.FloatSlider(min=0,max=14,step=0.25,value=7),
         I=widgets.FloatSlider(min=0,max=0.5,step=0.02,value=0.1));

interactive(children=(Text(value='C00022 = C00084 + C00011', description='plaintext'), FloatSlider(value=7.0, …

In [12]:
e2 = ComponentContribution(pH=7.0, ionic_strength=0.1)
rxn = reaction_matcher.match('C00022 = C00084 + C00011')

In [14]:
dG0, dGOp = e2.dG0_prime(rxn)

In [15]:
dG0

-26.754549493947422